In [5]:
import os
from pathlib import Path
import numpy as np
import json
import ultralytics
from ultralytics.data.converter import convert_coco

In [2]:
# Convert filenames in COCO
rel_coco_dir = "data/COCO_original"
coco_labels_path = f"{rel_coco_dir}/result.json"
coco_labels_path_filtered = f"data/COCO/result.json"
img_reldir = f"images"
with open(coco_labels_path, "r") as f:
    coco_data = json.load(f)
for image in coco_data["images"]:
    path = Path(image["file_name"])
    filename = path.name
    image["file_name"] = str(Path(img_reldir) / filename)

with open(coco_labels_path_filtered, "w") as f:
    json.dump(coco_data, f, indent=4)


In [9]:
# Make train test split
train_coco = {"images": [], "annotations": [], "categories": coco_data["categories"], "info": coco_data["info"]}
test_coco = {"images": [], "annotations": [], "categories": coco_data["categories"], "info": coco_data["info"]}

img_ids = [img["id"] for img in coco_data["images"]]
img_ids_shuffled = np.random.permutation(img_ids)
TRAIN_PORTION = 0.9
train_size = int(TRAIN_PORTION * len(img_ids))
train_ids = img_ids_shuffled[:train_size]
test_ids = img_ids_shuffled[train_size:]

for image in coco_data["images"]:
    if image["id"] in train_ids:
        train_coco["images"].append(image)
    else:
        test_coco["images"].append(image)

for annot in coco_data["annotations"]:
    if annot["image_id"] in train_ids:
        train_coco["annotations"].append(annot)
    else:
        test_coco["annotations"].append(annot)

In [ ]:
for image in train_coco["images"]:
    filename = Path(image["file_name"])
    

'images/78d5da2a-WhatsApp_Image_2025-11-26_at_17.47.40.jpeg'

In [16]:
model = ultralytics.YOLO(model="yolo11n.pt")

model.train(
    data="data/YOLO/config.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
)

New https://pypi.org/project/ultralytics/8.3.233 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.232 🚀 Python-3.13.9 torch-2.7.1+cu126 CUDA:0 (NVIDIA GeForce RTX 3090, 24115MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/YOLO/config.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, o

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f228af54440>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 